# Retrieve NetCDF and model gridded climate time-series for a watershed

### Case study:  the Sauk-Suiattle Watershed
<img src="http://www.sauk-suiattle.com/images/Elliott.jpg" 
style="float:right;width:150px;padding:20px">

### Use this Jupyter Notebook to:
    1. Prepare computing environment
    2. Get list of grid cells
    3. NetCDF retrieval and clipping to a spatial extent
    4. Extract NetCDF metadata and convert NetCDFs to 1D ASCII time-series files
    5. Visualize the average monthly total precipitations
    6. Apply summary values as modeling inputs
    7. Visualize modeling outputs
    8. Save results in a new HydroShare resource

##### For inquiries about functions, please refer to https://github.com/freshwater-initiative/Observatory 

<br/><br/><br/>
<img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png"
style="float:right;width:150px;padding:20px">

<br/><br/>
### Special thanks to Nicoleta Cristea and Jeff Keck for their contribution to this work
#### This data is compiled to digitally observe the watersheds, powered by HydroShare. <br/>Provided by the Watershed Dynamics Group, Dept. of Civil and Environmental Engineering, University of Washington

## 1.  Prepare computing environment

Here, we import ogh and several requisite libraries. Then, map directories that will be used.

In [ ]:
%%time
# If OGH is not installed, uncomment and run the line below

#!conda install -c conda-forge ogh xarray=0.11.2 --yes

Initialize data analysis libraries

In [ ]:
# silencing warning
import warnings
warnings.filterwarnings("ignore")

import os, pandas as pd, numpy as np
import ogh
from ogh import oxl
from ecohydrology_model_functions import run_ecohydrology_model, plot_results
from landlab import imshow_grid, CLOSED_BOUNDARY
import matplotlib.pyplot as plt
%matplotlib inline

Establish a secure connection with HydroShare REST API by initializing the class from the utilities folder. This command also sets environment variables that will be useful for saving work back to HydroShare.

In [ ]:
from utilities import hydroshare
hs=hydroshare.hydroshare()
homedir = os.getcwd()

# map modeling input params
InputFile = os.path.join(homedir,'ecohyd_inputs.yaml')

# initialize ogh metadata
meta_file = dict(ogh.ogh_meta())

# initialize list of outputs
files=[]

In [ ]:
homedir

## 2. Get list of grid cells

For visualization purposes, we will also remap the study site shapefile, which is stored in HydroShare at the following url: https://www.hydroshare.org/resource/c532e0578e974201a0bc40a37ef2d284/. Since the shapefile was previously migrated, we can select 'N' for no overwriting.

In the usecase1 notebook, the treatgeoself function identified the gridded cell centroid coordinates that overlap with our study site. These coordinates were documented within the mapping file, which will be remapped here. In the usecase2 notebook, the downloaded files were cataloged within the mapping file, so we will use the mappingfileSummary function to characterize the files available for Sauk-Suiattle for each gridded data product.

In [ ]:
"""
1/16-degree Gridded cell centroids
"""
# List of available data
hs.getResourceFromHydroShare('ef2d82bf960144b4bfb1bae6242bcc7f')
NAmer = hs.content['NAmer_dem_list.shp']


"""
Sauk
"""
# Watershed extent
hs.getResourceFromHydroShare('c532e0578e974201a0bc40a37ef2d284')
sauk = hs.content['wbdhub12_17110006_WGS84_Basin.shp']

# reproject the shapefile into WGS84
ogh.reprojShapefile(sourcepath=sauk)

### Summarize the target grid cells within a watershed

In [ ]:
%%time

# map the mappingfiles from usecase1
mappingfile1=ogh.treatgeoself(shapefile=sauk, NAmer=NAmer, buffer_distance=0.06,
                              mappingfile=os.path.join(homedir,'Sauk_mappingfile.csv'))

files.append(mappingfile1)

## 3.  NetCDF retrieval and clipping to a spatial extent

This section performs data retrieval and clips the datafile to the bounding box for your watershed. You will source datafiles from the Dr. Ben Livneh's 2013 publication for interpolated meteorological observations.

ftp://livnehpublicstorage.colorado.edu/public/Livneh.2013.CONUS.Dataset/Meteorology.nc.v.1.2.1915.2011.bz2/

In [ ]:
help(oxl.get_x_dailymet_Livneh2013_raw)

The function get_x_dailymet_livneh2013_raw retrieves and clips NetCDF files archived within the University of Colorado Boulder repository. This archive contains daily data captured for the Continental US, stored in monthly files.

We want to consider meteorology for Sauk-Suiattle watershed with data representing 1970. In the code chunk below, 6 parallel workers will be initialized to distribute the tasks. Each worker will be tasked to ftp request a file, clip the netcdf file to the provided spatial bounding box (in WGS84 standard projection), then return the location of the spatially clipped files. That is one task.

Provide the home and subdirectory as inputs for where the cropped NetCDF files will be stored, the number of workers to carry out the tasks, and the start and end dates of interest.

In [ ]:
%%time
maptable, nstations = ogh.mappingfileToDF(mappingfile1, summary=True)
spatialbounds = {'minx':maptable.LONG_.min(), 'maxx':maptable.LONG_.max(),
                 'miny':maptable.LAT.min(), 'maxy':maptable.LAT.max()}

outputfiles = oxl.get_x_dailymet_Livneh2013_raw(homedir=homedir,
                                                subdir='livneh2013/Daily_MET_1970_1970/raw_netcdf',
                                                spatialbounds=spatialbounds,
                                                nworkers=6,
                                                start_date='1970-01-01', end_date='1970-12-31')

files.extend(outputfiles)

## 4. Extract NetCDF metadata and convert NetCDFs to 1D ASCII time-series files

Here, convert the netCDF files into daily ascii time-series files for each grid cell location. For the 99 gridded data files intended, a 1D time-series ASCII will be created. The NetCDF metadata will be extracted to the meta_file dictionary using the catalog_label as the dictionary key.

Provide the home and subdirectory where the ASCII files will be stored, the directory for the source netCDF files, and the mapping file to which the resulting ASCII files will be cataloged. Also, provide the Pandas Datetime code for the frequency of the data time steps. Finally, provide the catalog label that will be used for the mapping file catalog and the metadata file label.

In [ ]:
%%time
outfilelist = oxl.netcdf_to_ascii(homedir=homedir, 
                                  subdir='livneh2013/Daily_MET_1970_1970/raw_ascii', 
                                  source_directory=os.path.join(homedir, 'livneh2013/Daily_MET_1970_1970/raw_netcdf'),
                                  mappingfile=mappingfile1,
                                  temporal_resolution='D',
                                  meta_file=meta_file,
                                  catalog_label='sp_dailymet_livneh_1970_1970')

# files.extend(outfilelist)

In [ ]:
sorted(meta_file.keys())

### Summarize file availability

In [ ]:
maptable, nstations = ogh.mappingfileToDF(mappingfile1, summary=True)
maptable.head()

In [ ]:
t1 = ogh.mappingfileSummary(listofmappingfiles = [mappingfile1], 
                            listofwatershednames = ['Sauk-Suiattle river'],
                            meta_file=meta_file)

t1

### Inspect the acquired metadata

In [ ]:
meta_file['sp_dailymet_livneh_1970_1970']['variable_list']

In [ ]:
# Save the metadata
metafile_path = os.path.join(homedir, 'test.json')
ogh.saveDictOfDf(dictionaryObject=meta_file, outfilepath=metafile_path)
files.append(metafile_path)

### Create a dictionary of climate variables for the long-term mean (ltm).
#### INPUT: gridded meteorology ASCII files located from the Sauk-Suiattle Mapping file. The locations are cataloged within the mappingfile, under the dataset catalog label. The gridclim_dict function will read in the files, and automatically compute the basic statistics for each variable.

#### OUTPUT: dictionary of dataframes where rows are temporal summaries and columns are spatial summaries

In [ ]:
%%time

ltm = ogh.gridclim_dict(mappingfile=mappingfile1,
                        metadata=meta_file,
                        dataset='sp_dailymet_livneh_1970_1970',
                        variable_list=['Prec','Tmax','Tmin'])

In [ ]:
sorted(ltm.keys())

In [ ]:
ltm['meanbymonth_Prec_sp_dailymet_livneh_1970_1970']

### Compute the total monthly and yearly precipitation, as well as the mean values across time and across stations. Append into the ltm dictionary.

In [ ]:
# extract metadata
dr = meta_file['sp_dailymet_livneh_1970_1970']

# compute sums and mean monthly an yearly sums
ltm = ogh.aggregate_space_time_sum(df_dict=ltm,
                                   suffix='Prec_sp_dailymet_livneh_1970_1970',
                                   start_date=dr['start_date'],
                                   end_date=dr['end_date'])

In [ ]:
# print the name of the analytical dataframes to see the new additions
# sorted(ltm.keys())

In [ ]:
# create the destination path for the dictionary of dataframes
ltm_sauk=os.path.join(homedir, 'ltm_1970_1970_sauk.json')
ogh.saveDictOfDf(dictionaryObject=ltm, outfilepath=ltm_sauk)
files.append(ltm_sauk)

## 5. Visualize the average monthly total precipitations

In [ ]:
# one highest elevation location
highE_ref = ogh.findCentroidCode(mappingfile=mappingfile1, colvar='ELEV', colvalue=2216)

# two lowest elevation locations
lowE_ref = ogh.findCentroidCode(mappingfile=mappingfile1, colvar='ELEV', colvalue=164)

# combine references together
reference_lines = highE_ref + lowE_ref
reference_lines

### Inspect the monthly time-series (in the wateryear schedule)

Each point in the boxplot distributions is a station's value

In [ ]:
%%time
# INPUT: dataframe with each month as a row and each station as a column. 
# OUTPUT: A png file that represents the distribution across stations (in Wateryear order)

ogh.renderValueInBoxplot(ltm['meanbymonthsum_Prec_sp_dailymet_livneh_1970_1970'],
                         outfilepath=os.path.join(homedir, 'totalMonthlyRainfall.png'), 
                         plottitle='Total monthly rainfall',
                         time_steps='month',
                         wateryear=True,
                         reference_lines=reference_lines,
                         ref_legend=True,
                         obs_legend=False,
                         value_name='Total daily precipitation (mm)',
                         cmap='seismic_r',
                         figsize=(6,6))

files.append(os.path.join(homedir, 'totalMonthlyRainfall.png'))

### Inspect the spatial distribution for a select month

In [ ]:
help(ogh.renderValuesInPoints)

In [ ]:
%%time
# INPUT: dataframe with each month as a row and each station as a column. 
# OUTPUT: A png file that represents the December spatial distribution of total precipitation

ogh.renderValuesInPoints(ltm['meanbymonthsum_Prec_sp_dailymet_livneh_1970_1970'], 
                         vardf_dateindex=12, 
                         shapefile=sauk, 
                         cmap='seismic_r',
                         gridcell_alpha=1,
                         outfilepath='test.png', 
                         plottitle='December total rainfall',
                         colorbar_label='Total monthly rainfall (mm)', 
                         figsize=(1.5,1.5))

files.append(os.path.join(homedir, 'test.png'))

## 6. Apply summary values as modeling inputs

### Convert the spatial bounding box to raster dimensions

In [ ]:
%%time
# compute the dimensions of the raster
minx2, miny2, maxx2, maxy2 = oxl.calculateUTMbounds(mappingfile=mappingfile1,
                                                    mappingfile_crs={'init':'epsg:4326'},
                                                    spatial_resolution=0.06250)

In [ ]:
print(minx2, miny2, maxx2, maxy2)

### Generate the raster

The Ecohydrology vegetation model within Landlab is run for a 2D landscape. The data input should be a raster array object configured for the amount of grid cell dimensions desired. Here, the raster dimensions are used to configure 1kmx1km grid cells.

In [ ]:
%%time

# generate a raster with 1kmx1km grid cells
raster, row_list, col_list = oxl.rasterDimensions(minx=minx2, miny=miny2, maxx=maxx2, maxy=maxy2, dx=1000, dy=1000)

# print the raster dimensions (column and rows)
raster.shape

### Higher resolution children of gridded cells 

Relate the parent grid cells (in the mappingfile) to higher resolution children nodes in the raster. Maintain this connection in the nodeXmap object.

In [ ]:
help(oxl.mappingfileToRaster)

In [ ]:
%%time

# landlab raster node crossmap to gridded cell id
nodeXmap, raster, m = oxl.mappingfileToRaster(mappingfile=mappingfile1, 
                                              spatial_resolution=0.06250,
                                              minx=minx2, miny=miny2, maxx=maxx2, maxy=maxy2,
                                              dx=1000, dy=1000)

In [ ]:
nodeXmap

### Visualize the elevation within the high resolutions children

In [ ]:
%%time

nodeXmap.fillna(nodeXmap['ELEV'].min()).plot(column='ELEV', figsize=(10,10), legend=True, cmap='terrain')

### Visualize the average monthly precipitation for December relative to all months

In [ ]:
%%time
# generate vector array of December monthly precipitation
prec_vector = oxl.rasterVector(vardf=ltm['meanbymonthsum_Prec_sp_dailymet_livneh_1970_1970'],
                               vardf_dateindex=12,
                               crossmap=nodeXmap,
                               nodata=-9999)

# close-off areas without data
raster.status_at_node[prec_vector==-9999] = CLOSED_BOUNDARY

fig =plt.figure(figsize=(10,10))
imshow_grid(raster,
            prec_vector,
            var_name='Monthly precipitation',
            var_units=meta_file['sp_dailymet_livneh_1970_1970']['variable_info']['Prec'].attrs['units'],
            color_for_closed='black',
            cmap='seismic_r',
            vmin=ltm['meanbymonthsum_Prec_sp_dailymet_livneh_1970_1970'].unstack().min(),
            vmax=ltm['meanbymonthsum_Prec_sp_dailymet_livneh_1970_1970'].unstack().max())

### Visualize average maximum daily temperature for December relative to all months

In [ ]:
tmax_vector = oxl.rasterVector(vardf=ltm['meanbymonth_Tmax_sp_dailymet_livneh_1970_1970'],
                               vardf_dateindex=12,
                               crossmap=nodeXmap,
                               nodata=-9999)

fig = plt.figure(figsize=(10,10))
imshow_grid(raster,
            tmax_vector,
            var_name='Daily maximum temperature',
            var_units=meta_file['sp_dailymet_livneh_1970_1970']['variable_info']['Tmax'].attrs['units'],
            color_for_closed='black',
            cmap='magma',
            vmax=ltm['meanbymonth_Tmax_sp_dailymet_livneh_1970_1970'].unstack().max(),
            vmin=ltm['meanbymonth_Tmax_sp_dailymet_livneh_1970_1970'].unstack().min())

### Visualize the average minimum daily temperature for December relative to all months

In [ ]:
tmin_vector = oxl.rasterVector(vardf=ltm['meanbymonth_Tmin_sp_dailymet_livneh_1970_1970'],
                               vardf_dateindex=12,
                               crossmap=nodeXmap,
                               nodata=-9999)

fig = plt.figure(figsize=(10,10))
imshow_grid(raster, 
            tmin_vector,
            var_name='Daily minimum temperature',
            var_units=meta_file['sp_dailymet_livneh_1970_1970']['variable_info']['Tmin'].attrs['units'],
            color_for_closed='black', 
            cmap='magma',
            vmax=ltm['meanbymonth_Tmin_sp_dailymet_livneh_1970_1970'].unstack().max(),
            vmin=ltm['meanbymonth_Tmin_sp_dailymet_livneh_1970_1970'].unstack().min())

### Visualize the average monthly precipitation for December, again, but using Google standard projection

In [ ]:
# convert a raster vector back to geospatial presentation
t2, t3 = oxl.rasterVectorToWGS(prec_vector, nodeXmap=nodeXmap, UTM_transformer=m)

t2.crs = {'init':'epsg:4326'}
t2.drop('raster_geom', axis=1).to_file(os.path.join(homedir,'hires_sauk.shp'))
t2.plot(column='value', figsize=(10,6), legend=True, cmap='seismic_r',
        vmin=ltm['meanbymonthsum_Prec_sp_dailymet_livneh_1970_1970'].unstack().min(), 
        vmax=ltm['meanbymonthsum_Prec_sp_dailymet_livneh_1970_1970'].unstack().max())

### Visualize the average monthly precipitation for December, again, but using the North Washington projection

In [ ]:
t2 = t2.to_crs({'init':'epsg:3857'})
t2.plot(column='value', figsize=(10,6), legend=True, cmap='seismic_r',
        vmin=ltm['meanbymonthsum_Prec_sp_dailymet_livneh_1970_1970'].unstack().min(), 
        vmax=ltm['meanbymonthsum_Prec_sp_dailymet_livneh_1970_1970'].unstack().max())

In [ ]:
# add in the shp file parts
files.extend([os.path.join(homedir, shpfile) for shpfile in os.listdir(homedir) if 'hires_sauk' in shpfile])

### Prepare the meteorological summary for 1970 as the model input vectors

In [ ]:
# configure the input vector with 15000 repetitions 
inputvectors = {'precip_met': np.tile(ltm['meandaily_Prec_sp_dailymet_livneh_1970_1970'], 15000),
                'Tmax_met': np.tile(ltm['meandaily_Tmax_sp_dailymet_livneh_1970_1970'], 15000),
                'Tmin_met': np.tile(ltm['meandaily_Tmin_sp_dailymet_livneh_1970_1970'], 15000)}

In [ ]:
%%time
# run ecohydrology model for 100000 storms

(VegType_low, yrs_low, debug_low) = run_ecohydrology_model(raster,
                                                           input_data=inputvectors,
                                                           input_file=InputFile,
                                                           synthetic_storms=False,
                                                           number_of_storms=100000,
                                                           pet_method='PriestleyTaylor')

## 7. Visualize modeling outputs

In [ ]:
%%time
plot_results(raster, VegType_low, yrs_low, yr_step=yrs_low-1)
plt.show()
plt.savefig(os.path.join(homedir,'grid_low.png'))
files.append(os.path.join(homedir,'grid_low.png'))

## 8. Save the results back into HydroShare

Create a new hydroshare resource to store the Geoprocessing output files listed within 'files'. First, define all of the required metadata for resource creation, i.e. *title*, *abstract*, *keywords*, *content files*. In addition, we must define the type of resource that will be created, in this case *compositeresource*.  

***Note:*** Make sure you save the notebook at this point, so that all notebook changes will be saved into the new HydroShare resource.

In [ ]:
### add the notebook
files.append(os.path.join(homedir, 'Observatory_usecase7_xmapLandlab.ipynb'))

### Total files and image to migrate

In [ ]:
len(files)

In [ ]:
# for each file downloaded onto the server folder, move to a new HydroShare Composite Resource
title = 'Computed spatial-temporal summaries for Sauk-Suiattle for 1970'
abstract = 'This resource contains the computed summaries for the Meteorology data from Livneh et al. 2013.'
keywords = ['Sauk-Suiattle', 'Livneh 2013', 'climate', 'hydromet', 'watershed', 'visualizations and summaries'] 
rtype = 'compositeresource'
parent_resource_id = homedir.replace('/data/contents','').rsplit('/',1)[1]

# create the new resource
resource_id = hs.createHydroShareResource(abstract,
                                          title,
                                          keywords=keywords,
                                          resource_type=rtype,
                                          content_files=files,
                                          derivedFromId=parent_resource_id,
                                          public=False)

Thanks for trying this notebook!

In [ ]:
# # # ### Once done, uncomment to empty the stored data ###
# # # 
# # #
# # # # Delete items in 'files' but skip this notebook
# # # for eachf in files[:-1]:
# # #     os.remove(eachf)
# # # 
# # # 
# # # import shutil
# # # shutil.rmtree(os.path.join(homedir, 'livneh2013'))
# # # 
# # # 